In [1]:
%%configure -f
{
    "conf": {
        "spark.app.name": "datavirus_timetable"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8044,application_1589299642358_2564,pyspark,busy,Link,Link,
8063,application_1589299642358_2587,pyspark,busy,Link,Link,
8082,application_1589299642358_2606,pyspark,idle,Link,Link,
8091,application_1589299642358_2615,pyspark,busy,Link,Link,
8095,application_1589299642358_2619,pyspark,busy,Link,Link,
8101,application_1589299642358_2625,pyspark,idle,Link,Link,
8103,application_1589299642358_2627,pyspark,idle,Link,Link,
8107,application_1589299642358_2631,pyspark,busy,Link,Link,
8108,application_1589299642358_2632,pyspark,idle,Link,Link,
8110,application_1589299642358_2635,pyspark,idle,Link,Link,


In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8148,application_1589299642358_2674,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8044,application_1589299642358_2564,pyspark,busy,Link,Link,
8063,application_1589299642358_2587,pyspark,busy,Link,Link,
8082,application_1589299642358_2606,pyspark,idle,Link,Link,
8091,application_1589299642358_2615,pyspark,busy,Link,Link,
8095,application_1589299642358_2619,pyspark,busy,Link,Link,
8101,application_1589299642358_2625,pyspark,idle,Link,Link,
8103,application_1589299642358_2627,pyspark,idle,Link,Link,
8107,application_1589299642358_2631,pyspark,busy,Link,Link,
8108,application_1589299642358_2632,pyspark,idle,Link,Link,
8110,application_1589299642358_2635,pyspark,idle,Link,Link,


In [33]:
import pyspark.sql.functions as f
from pyspark.sql import Window

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
trips = spark.read.orc("/data/sbb/timetables/orc/trips/")
calendar = spark.read.orc("/data/sbb/timetables/orc/calendar/")
routes = spark.read.orc("/data/sbb/timetables/orc/routes/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
trips.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+--------------------+------------------+---------------+------------+
|   route_id|service_id|             trip_id|     trip_headsign|trip_short_name|direction_id|
+-----------+----------+--------------------+------------------+---------------+------------+
|1-1-C-j19-1|  TA+b0001|5.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            108|           1|
+-----------+----------+--------------------+------------------+---------------+------------+
only showing top 1 row

In [7]:
calendar.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|
+----------+------+-------+---------+--------+------+--------+------+
|  TA+b0nx9|  true|   true|     true|    true|  true|   false| false|
+----------+------+-------+---------+--------+------+--------+------+
only showing top 1 row

In [8]:
routes.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------------+---------------+----------+----------+
|   route_id|agency_id|route_short_name|route_long_name|route_desc|route_type|
+-----------+---------+----------------+---------------+----------+----------+
|11-40-j19-1|      801|             040|               |       Bus|       700|
+-----------+---------+----------------+---------------+----------+----------+
only showing top 1 row

In [9]:
# select all services that are scheduled on mondays
monday_service_id = (
    calendar
    .where(calendar.monday == True)
    .select('service_id')
    .dropDuplicates()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# keep only trips that are scheduled on mondays
monday_trips = (
    trips
    .join(f.broadcast(monday_service_id), 'service_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
# add route information to trips
route_trips = (
    trips
    .join(routes, 'route_id')
    
    # drop all transports that are not of type:
    # InterCity, InterRegio, Regionalzug, S-Bahn, Bus, Tram
    .where(routes.route_type.isin(['102', '103', '106', '400', '700', '900']))
    
    # create id that is used to merge with ist data
    .withColumn(
        'id',
        f.when(routes.route_type.isin(['700', '900']),
            f.concat_ws(':', f.lit('85'), routes.agency_id.cast('int').cast('string'), routes.route_short_name.cast('int').cast('string'))
        ).otherwise(
            f.concat_ws(':', f.lit('85'), routes.agency_id.cast('int').cast('string'), trips.trip_short_name.cast('int').cast('string'))
        )
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
probability = spark.read.orc('/user/datavirus/probability.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
route_probability = (
    route_trips
    .join(probability, 'id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
stop_time = spark.read.orc("/data/sbb/timetables/orc/stop_times/")

stop_time = (
    stop_time
    .select(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])
    .filter(
        (stop_time.departure_time >= '05:00:00')
        & (stop_time.departure_time <= '21:00:00')
        & (stop_time.arrival_time >= '05:00:00')
        & (stop_time.arrival_time <= '21:00:00') 
    )
    .withColumn('station_id', stop_time.stop_id.substr(0, 7))
    .drop('stop_id')
    
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
stations = spark.read.csv("../data/zurich_stations_ids.csv")
stations = (
    stations
    .select(stations._c0.alias('station_id'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
stop_filtered = (
    stop_time
    .join(f.broadcast(stations), 'station_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
stop_keep = (
    stop_filtered
    .groupBy('trip_id')
    .count()
    .filter('count > 1')
    .select('trip_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:
trip_window = Window.partitionBy('trip_id').orderBy(f.asc('arrival_time'))
stop_sequence = f.rank().over(trip_window)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [57]:
stop_resequenced = (
    stop_filtered
    .join(stop_keep, 'trip_id')
    .withColumn('stop_sequence', stop_sequence)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [71]:
stop_resequenced.write.format('orc').save('/user/datavirus/stop_resequenced.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
stop_resequenced = spark.read.orc('/user/datavirus/stop_resequenced.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [85]:
departure = stop_resequenced.alias('departure')
arrival = stop_resequenced.withColumn('stop_sequence', stop_resequenced.stop_sequence - 1).alias('arrival')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
joined = (
    departure
    .join(arrival, ['trip_id', 'stop_sequence'])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
joined.show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
departure.join(arrival, ['trip_id', 'stop_sequence']).select(departure.trip_id, departure.departure_time)

In [ ]:

new_counting =timetable.select('*', trip_rank).alias('new_counting').orderBy('trip_id','arrival_time').fillna(0)# Remove trips which only appear once (meaning the go to/come from stops outside Zurich)
ids_counts = begin.groupBy('trip_id').count()
ids_trips = ids_counts.where(ids_counts['count']>1).select('trip_id').distinct()
df = new_counting.join(ids_trips, "trip_id").orderBy('trip_id','arrival_time','new_stop')# Match each stop with the next stop (to transform trips to connections)
to_station = df.drop('departure_time','stop_sequence').withColumn('new_stop', df.new_stop -1).alias('to_station')
from_station = df.drop('arrival_time','stop_sequence').join(to_station, on=['new_stop','trip_id']).orderBy('trip_id','new_stop')# Write schedule in "connection" format (two stops per row)
schedule = from_station.toDF("Stop_sequence","Trip_ID","Start_Time","Start_Station","Stop_Time","Stop_Station")
schedule.write.format("orc").save("/user/datavirus/schedule_zurich.orc", mode='overwrite')

In [ ]:
raw_connection = (
    
)

In [27]:
departure = (
    stop_time
    .join(route_probability, 'station_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
stations = spark.read.csv("../data/zurich_stations_ids.csv")
stations = (
    stations
    .select(stations._c0.alias('stop_id'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
sbb = spark.read.orc('/data/sbb/orc/istdaten')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
sbb.where(sbb.fahrt_bezeichner.substr(0, 10) == '85:151:2').show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------------+------------+-------------+----------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+----------------------+----------------+-------------------+------------------+----------------+-------------------+------------------+-------------+
|betriebstag|fahrt_bezeichner|betreiber_id|betreiber_abk|betreiber_name  |produkt_id|linien_id|linien_text|umlauf_id|verkehrsmittel_text|zusatzfahrt_tf|faellt_aus_tf|bpuic  |haltestellen_name     |ankunftszeit    |an_prognose        |an_prognose_status|abfahrtszeit    |ab_prognose        |ab_prognose_status|durchfahrt_tf|
+-----------+----------------+------------+-------------+----------------+----------+---------+-----------+---------+-------------------+--------------+-------------+-------+----------------------+----------------+-------------------+------------------+----------------+-------------------+------------------+-------------+
|03.09.2018 |85:94:3110:002 

In [25]:
route_trips.where(route_trips.route_type.isNull()).count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0

In [13]:
route_trips.dropDuplicates(['route_type']).show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------+-----------------------+-------------------------+---------------+------------+---------+----------------+---------------+-------------+----------+------------+-------------+
|route_id      |service_id|trip_id                |trip_headsign            |trip_short_name|direction_id|agency_id|route_short_name|route_long_name|route_desc   |route_type|ist_trip_id |ist_line_id  |
+--------------+----------+-----------------------+-------------------------+---------------+------------+---------+----------------+---------------+-------------+----------+------------+-------------+
|20-20-A-j19-1 |TA+b080b  |1.TA.20-20-A-j19-1.1.H |Castione-Arbedo          |31211          |0           |11       |20              |               |Auoreisezug  |1700      |85:11:31211 |85:11:20     |
|24-90-j19-1   |TA+b0001  |1.TA.24-90-j19-1.1.H   |Brig                     |1705           |0           |11       |90              |               |InterRegio   |103       |85:11:1705  |85:11

In [22]:
route_trips.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+---------------------+------------------+---------------+------------+---------+----------------+---------------+----------+----------+----------------+----------------+
|route_id   |service_id|trip_id              |trip_headsign     |trip_short_name|direction_id|agency_id|route_short_name|route_long_name|route_desc|route_type|train_identifier|other_identifier|
+-----------+----------+---------------------+------------------+---------------+------------+---------+----------------+---------------+----------+----------+----------------+----------------+
|1-1-C-j19-1|TA+b0001  |5.TA.1-1-C-j19-1.3.R |Zofingen, Altachen|108            |1           |723      |1               |               |Bus       |700       |85:723:108      |85:723:1        |
|1-1-C-j19-1|TA+b0001  |7.TA.1-1-C-j19-1.3.R |Zofingen, Altachen|112            |1           |723      |1               |               |Bus       |700       |85:723:112      |85:723:1        |
|1-1-C-j19-1|TA+b0001  |9.TA.1

In [23]:
route_trips.dropDuplicates(['route_desc']).show(100, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+----------+------------------------+-------------------------+---------------+------------+---------+----------------+---------------+-------------+----------+----------------+----------------+
|route_id      |service_id|trip_id                 |trip_headsign            |trip_short_name|direction_id|agency_id|route_short_name|route_long_name|route_desc   |route_type|train_identifier|other_identifier|
+--------------+----------+------------------------+-------------------------+---------------+------------+---------+----------------+---------------+-------------+----------+----------------+----------------+
|57-3-Y-j19-1  |TA+b0030  |9.TA.57-3-Y-j19-1.9.H   |Zürich HB                |9211           |0           |11       |3-Y             |               |TGV          |102       |85:11:9211      |85:11:3-Y       |
|40-12-Y-j19-1 |TA+b000n  |14.TA.40-12-Y-j19-1.7.H |Como S. Giovanni         |315            |0           |11       |12-Y            |               |Eurocity  

In [ ]:
@f.udf
def trip_id(id_):
    parts = id_.split(':')
    if len(parts) > 1:
        return parts[1]
    else:
        return None

In [25]:
stop_time = spark.read.orc("/data/sbb/timetables/orc/stop_times/").repartition(10, 'stop_id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
stop_time.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- trip_id: string (nullable = true)
 |-- arrival_time: string (nullable = true)
 |-- departure_time: string (nullable = true)
 |-- stop_id: string (nullable = true)
 |-- stop_sequence: short (nullable = true)
 |-- pickup_type: byte (nullable = true)
 |-- drop_off_type: byte (nullable = true)

In [27]:
stop_time.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|             trip_id|arrival_time|departure_time|    stop_id|stop_sequence|pickup_type|drop_off_type|
+--------------------+------------+--------------+-----------+-------------+-----------+-------------+
|1.TA.1-1-B-j19-1.1.R|    04:28:00|      04:28:00|8500021:0:5|            3|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:35:00|      04:35:00|8500313:0:2|            6|          0|            0|
|1.TA.1-1-B-j19-1.1.R|    04:52:00|      04:52:00|8500304:0:2|           11|          0|            0|
|25.TA.1-1-B-j19-1...|    05:58:00|      05:58:00|8500021:0:5|            3|          0|            0|
|25.TA.1-1-B-j19-1...|    06:05:00|      06:05:00|8500313:0:2|            6|          0|            0|
|25.TA.1-1-B-j19-1...|    06:22:00|      06:22:00|8500304:0:2|           11|          0|            0|
|28.TA.1-1-B-j19-1...|    06:28:00|      06:28:00|8500021:0:5|           

In [28]:
@f.udf
def parent_station_id(stop_id):
    return stop_id.split(':')[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
stop_time = (
    stop_time
    .select(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])
    .withColumn('stop_id', parent_station_id(stop_time.stop_id))
    .filter(
        (stop_time.departure_time >= '05:00:00')
        & (stop_time.departure_time <= '21:00:00')
        & (stop_time.arrival_time >= '05:00:00')
        & (stop_time.arrival_time <= '21:00:00')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
stop_time.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|
+--------------------+------------+--------------+-------+-------------+
|25.TA.1-1-B-j19-1...|    05:58:00|      05:58:00|8500021|            3|
|25.TA.1-1-B-j19-1...|    06:05:00|      06:05:00|8500313|            6|
|25.TA.1-1-B-j19-1...|    06:22:00|      06:22:00|8500304|           11|
|28.TA.1-1-B-j19-1...|    06:28:00|      06:28:00|8500021|            3|
|28.TA.1-1-B-j19-1...|    06:35:00|      06:35:00|8500313|            6|
|28.TA.1-1-B-j19-1...|    06:52:00|      06:52:00|8500304|           11|
|45.TA.1-1-B-j19-1...|    09:28:00|      09:28:00|8500021|            3|
|45.TA.1-1-B-j19-1...|    09:35:00|      09:35:00|8500313|            6|
|45.TA.1-1-B-j19-1...|    09:52:00|      09:52:00|8500304|           11|
|53.TA.1-1-B-j19-1...|    10:28:00|      10:28:00|8500021|            3|
+--------------------+------------+--------------+-

In [31]:
stations = spark.read.csv("../data/zurich_stations_ids.csv")
stations = (
    stations
    .select(stations._c0.alias('stop_id'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
stop_ids = [row['stop_id'] for row in stations.collect()]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [38]:
stop_ids = spark.sparkContext.broadcast(stop_ids)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
filtered = stop_time.where(stop_time.stop_id.isin(stop_ids.value))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
filtered.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+------------+--------------+-------+-------------+
|             trip_id|arrival_time|departure_time|stop_id|stop_sequence|
+--------------------+------------+--------------+-------+-------------+
|1.TA.1-11-B-j19-1...|    06:16:00|      06:16:00|8594304|            3|
|2.TA.1-11-B-j19-1...|    06:46:00|      06:46:00|8594304|            3|
|3.TA.1-11-B-j19-1...|    07:16:00|      07:16:00|8594304|            3|
|4.TA.1-11-B-j19-1...|    07:46:00|      07:46:00|8594304|            3|
|5.TA.1-11-B-j19-1...|    08:16:00|      08:16:00|8594304|            3|
|6.TA.1-11-B-j19-1...|    08:46:00|      08:46:00|8594304|            3|
|7.TA.1-11-B-j19-1...|    09:16:00|      09:16:00|8594304|            3|
|8.TA.1-11-B-j19-1...|    09:46:00|      09:46:00|8594304|            3|
|9.TA.1-11-B-j19-1...|    10:16:00|      10:16:00|8594304|            3|
|10.TA.1-11-B-j19-...|    10:46:00|      10:46:00|8594304|            3|
+--------------------+------------+--------------+-

In [41]:
timetable = (
    filtered
    .orderBy(['trip_id', 'stop_sequence'])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
timetable.write.format("csv").save("/user/gflueck/timetable_zurich.csv", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
(
    stop_time
    .groupBy(['trip_id', 'arrival_time'])
    .agg(
        f.collect_list(f.col('stop_sequence')).alias('sequence'), 
        f.collect_list(f.col('stop_id')).alias('stops'), 
        f.countDistinct(f.col('stop_id')).alias('count'),
    )
    .where(f.col('count') > 5)
).show(10, False)

In [14]:
departure = stop_time.alias('departure')
arrival = stop_time.withColumn('stop_sequence', stop_time.stop_sequence - 1).alias('arrival')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
timetable = (
    departure
    .join(arrival, ['trip_id', 'stop_sequence'])
    .select(
        departure.trip_id,
        departure.stop_sequence,
        departure.stop_id.alias('start_id'),
        departure.departure_time.alias('start_time'),
        arrival.arrival_time.alias('stop_time'),
        arrival.stop_id.alias('stop_id')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
timetable.write.format("csv").save("/user/gflueck/timetable.csv", mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…